## ETL Process for Synthetic Retail Data

In [2]:
!pip install faker



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 1.7 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.9 MB 2.3 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 2.3 MB/s eta 0:00:00


In [3]:
#Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import logging
from faker import Faker
from datetime import datetime, timedelta

In [5]:
# ----------------------------
# Setup logging
# ----------------------------
import pandas as pd
import numpy as np
import sqlite3
import logging
from datetime import datetime, timedelta
from faker import Faker  # make sure you have installed this: pip install faker

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ----------------------------
# Generate synthetic data
# ----------------------------
def generate_synthetic_data(num_rows=1000, num_customers=100):
    fake = Faker()
    Faker.seed(42)
    np.random.seed(42)

    customer_ids = [f"CUST{str(i).zfill(3)}" for i in range(1, num_customers+1)]
    countries = ["USA", "UK", "Germany", "France", "Canada", "Australia", "Japan", "Brazil"]
    products = [f"Product_{i}" for i in range(1, 21)]
    categories = ["Electronics", "Clothing", "Home", "Toys"]

    data = []
    for _ in range(num_rows):
        cust_id = np.random.choice(customer_ids)
        country = np.random.choice(countries)
        product = np.random.choice(products)
        category = np.random.choice(categories)
        quantity = np.random.randint(1, 51)
        unit_price = np.random.randint(1, 101)
        invoice_date = datetime.today() - timedelta(days=np.random.randint(0, 730))  # last 2 years
        data.append([cust_id, country, product, category, quantity, unit_price, invoice_date])

    df = pd.DataFrame(data, columns=["CustomerID", "Country", "Product", "Category", "Quantity", "UnitPrice", "InvoiceDate"])
    return df

# ----------------------------
# ETL Process
# ----------------------------
def etl_process():
    logging.info("Starting ETL process...")

    # Extract
    logging.info("Generating synthetic data...")
    df = generate_synthetic_data()
    logging.info(f"Generated {len(df)} rows.")

    # ✅ Save to CSV before transformations
    df.to_csv("retail_sales.csv", index=False)
    logging.info("Saved raw data to retail_sales.csv")

    # Handle missing values
    df.dropna(inplace=True)

    # Ensure correct data types
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

    # Transform
    logging.info("Transforming data...")
    df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]  # remove outliers
    df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

    # Filter last year
    cutoff_date = datetime(2024, 8, 12)
    df_last_year = df[df["InvoiceDate"] >= cutoff_date]
    logging.info(f"Filtered {len(df_last_year)} rows for the last year.")

    # Create customer dimension
    customer_dim = df.groupby(["CustomerID", "Country"], as_index=False).agg(
        TotalPurchases=("TotalSales", "sum"),
        Orders=("CustomerID", "count")
    )

    # Create time dimension
    time_dim = pd.DataFrame({
        "InvoiceDate": pd.to_datetime(df["InvoiceDate"].unique())
    })
    time_dim["Year"] = time_dim["InvoiceDate"].dt.year
    time_dim["Quarter"] = time_dim["InvoiceDate"].dt.quarter
    time_dim["Month"] = time_dim["InvoiceDate"].dt.month
    time_dim["Day"] = time_dim["InvoiceDate"].dt.day
    time_dim.reset_index(drop=True, inplace=True)
    time_dim["TimeID"] = time_dim.index + 1

    # Map TimeID back to sales fact
    df["TimeID"] = df["InvoiceDate"].map(
        time_dim.set_index("InvoiceDate")["TimeID"]
    )

    # Load
    logging.info("Loading data into SQLite database...")
    conn = sqlite3.connect("retail_dw.db")
    customer_dim.to_sql("CustomerDim", conn, if_exists="replace", index=False)
    time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)

    fact_sales = df[["CustomerID", "Product", "Category", "Quantity", "UnitPrice", "TotalSales", "TimeID"]]
    fact_sales.to_sql("SalesFact", conn, if_exists="replace", index=False)

    conn.close()
    logging.info("ETL process completed successfully.")

# ----------------------------
# Run ETL
# ----------------------------
if __name__ == "__main__":
    etl_process()


2025-08-12 17:25:57,077 - INFO - Starting ETL process...
2025-08-12 17:25:57,080 - INFO - Generating synthetic data...
2025-08-12 17:25:57,237 - INFO - Generated 1000 rows.
2025-08-12 17:25:57,310 - INFO - Saved raw data to retail_sales.csv
2025-08-12 17:25:57,327 - INFO - Transforming data...
2025-08-12 17:25:57,343 - INFO - Filtered 498 rows for the last year.
2025-08-12 17:25:57,393 - INFO - Loading data into SQLite database...
2025-08-12 17:25:57,494 - INFO - ETL process completed successfully.
